# Daniel Krasnov

## Research question/interestsDoes where you are in the world (country/city/longitude and latitude) affect the quality of food available to you (based on rating column), the price (based on average price for two column), and the type of food (based on cuisine column)?.

## Analysis Pipeline
### Loading Data

In [98]:
#Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns#

#reading in CSVs (Don't need the json data)
zomatoCSV = pd.read_csv("..\\data\\raw\\zomato.csv" , encoding = "maclatin2")
countryCodesCSV = pd.read_excel("..\\data\\raw\\Country-Code.xlsx")

#Getting a sense of my data
print(zomatoCSV.head())
print(countryCodesCSV.head())

   Restaurant ID         Restaurant Name  Country Code              City  \
0        6317637        Le Petit Souffle           162       Makati City   
1        6304287        Izakaya Kikufuji           162       Makati City   
2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
3        6318506                    Ooma           162  Mandaluyong City   
4        6314302             Sambo Kojin           162  Mandaluyong City   

                                             Address  \
0  Third Floor, Century City Mall, Kalayaan Avenu...   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   

                                     Locality  \
0   Century City Mall, Poblacion, Makati City   
1  Little Tokyo, Legaspi Village, Makati City   
2  Edsa Shangri-La, Ortigas, Mandaluyong City   
3      SM 

### Cleaning/Wrangling Data
Immedaitely I can see some very interesting data, but there are a few columns I know I will not be needing. "Zomato uses a 5-point classroom-style grading model, where the distribution of scores in every city is normalized" ([Zomato Blog](https://www.zomato.com/blog/urbanspoon-to-zomato-the-5-point-rating-system-3)). With this in mind, something like rating text or rating color won't be particularily useful since aggregate rating is present. For now I keep votes since which restaurants are so good (or so bad) that they encourage people to vote will be interesting information. Locality verbose and address seems like overkill when I already have so much other location data so we'll drop these columns.

In [12]:
zomatoCSV = zomatoCSV.drop(['Address','Rating color','Rating text', 'Locality Verbose'],axis = 'columns')

,Restaurant ID,Restaurant Name,Country Code,City,Locality,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Century City Mall, Poblacion, Makati City",121.027535,14.565443,"French, Japanese, Desserts",1100,Botswana Pula(P),Yes,No,No,No,3,4.8,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, Legaspi Village, Makati City",121.014101,14.553708,Japanese,1200,Botswana Pula(P),Yes,No,No,No,3,4.5,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, Ortigas, Mandaluyong City",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000,Botswana Pula(P),Yes,No,No,No,4,4.4,270
3,6318506,Ooma,162,Mandaluyong City,"SM Megamall, Ortigas, Mandaluyong City",121.056475,14.585318,"Japanese, Sushi",1500,Botswana Pula(P),No,No,No,No,4,4.9,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"SM Megamall, Ortigas, Mandaluyong City",121.057508,14.584450,"Japanese, Korean",1500,Botswana Pula(P),Yes,No,No,No,4,4.8,229


It'd be nice to see countrys and cities set up in the indices to see where restaurants are easier. Merging countryCodes with zomatoCSV and setting the country as the outer index and city as the inner index will solve this.

In [102]:
cleaned_df = pd.merge(zomatoCSV,countryCodesCSV, how = 'left', on='Country Code')
cleaned_df = cleaned_df.set_index(['Country','City']) \
                       .drop('Country Code', axis = 'columns')
cleaned_df

Restaurant ID           Restaurant Name  \
Country     City                                                        
Phillipines Makati City             6317637          Le Petit Souffle   
            Makati City             6304287          Izakaya Kikufuji   
            Mandaluyong City        6300002    Heat - Edsa Shangri-La   
            Mandaluyong City        6318506                      Ooma   
            Mandaluyong City        6314302               Sambo Kojin   
...                                     ...                       ...   
Turkey      ŘŃstanbul               5915730              NamlŘĪ Gurme   
            ŘŃstanbul               5908749             Ceviz AŘŰacŘĪ   
            ŘŃstanbul               5915807                     Huqqa   
            ŘŃstanbul               5916112              AĀŰĀŰk Kahve   
            ŘŃstanbul               5927402  Walter's Coffee Roastery   

                                                                        Address  \
Country     City                                                                  
Phillipines Makati City       Third Floor, Century City Mall, Kalayaan Avenu...   
            Makati City       Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
            Mandaluyong City  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
            Mandaluyong City  Third Floor, Mega Fashion Hall, SM Megamall, O...   
            Mandaluyong City  Third Floor, Mega Atrium, SM Megamall, Ortigas...   
...                                                                         ...   
Turkey      ŘŃstanbul         KemankeĀŰ Karamustafa PaĀŰa Mahallesi, RŘĪhtŘĪ...   
            ŘŃstanbul         KoĀŰuyolu Mahallesi, Muhittin ŪžstŪ_ndaŘŰ Cadd...   
            ŘŃstanbul         KuruŪ_eĀŰme Mahallesi, Muallim Naci Caddesi, N...   
            ŘŃstanbul         KuruŪ_eĀŰme Mahallesi, Muallim Naci Caddesi, N...   
            ŘŃstanbul         CafeaŘŰa Mahallesi, BademaltŘĪ Sokak, No 21/B,...   

                                                                Locality  \
Country     City                                                           
Phillipines Makati City        Century City Mall, Poblacion, Makati City   
            Makati City       Little Tokyo, Legaspi Village, Makati City   
            Mandaluyong City  Edsa Shangri-La, Ortigas, Mandaluyong City   
            Mandaluyong City      SM Megamall, Ortigas, Mandaluyong City   
            Mandaluyong City      SM Megamall, Ortigas, Mandaluyong City   
...                                                                  ...   
Turkey      ŘŃstanbul                                           KarakŪ_y   
            ŘŃstanbul                                          KoĀŰuyolu   
            ŘŃstanbul                                        KuruŪ_eĀŰme   
            ŘŃstanbul                                        KuruŪ_eĀŰme   
            ŘŃstanbul                                               Moda   

                                                               Locality Verbose  \
Country     City                                                                  
Phillipines Makati City       Century City Mall, Poblacion, Makati City, Mak...   
            Makati City       Little Tokyo, Legaspi Village, Makati City, Ma...   
            Mandaluyong City  Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...   
            Mandaluyong City  SM Megamall, Ortigas, Mandaluyong City, Mandal...   
            Mandaluyong City  SM Megamall, Ortigas, Mandaluyong City, Mandal...   
...                                                                         ...   
Turkey      ŘŃstanbul                                       KarakŪ_y, ŘŃstanbul   
            ŘŃstanbul                                      KoĀŰuyolu, ŘŃstanbul   
            ŘŃstanbul                                    KuruŪ_eĀŰme, ŘŃstanbul   
            ŘŃstanbul                                    KuruŪ_eĀŰme, ŘŃstanbul   
            ŘŃs

Now it would be useful to see what countries we have in this data set, taking a look at our unique indices we have representation from every continent! This will be extremly helpful in answering the research question since we have such diverse representation.

In [103]:
print(cleaned_df.index.unique(0))

Index(['Phillipines', 'Brazil', 'United States', 'Australia', 'Canada',
       'Singapore', 'UAE', 'India', 'Indonesia', 'New Zealand',
       'United Kingdom', 'Qatar', 'South Africa', 'Sri Lanka', 'Turkey'],
      dtype='object', name='Country')


Missing data is up next, to get a sense of what we're dealing with we run the following:

In [105]:
cleaned_df.isna().any()

Restaurant ID           False
Restaurant Name         False
Address                 False
Locality                False
Locality Verbose        False
Longitude               False
Latitude                False
Cuisines                 True
Average Cost for two    False
Currency                False
Has Table booking       False
Has Online delivery     False
Is delivering now       False
Switch to order menu    False
Price range             False
Aggregate rating        False
Rating color            False
Rating text             False
Votes                   False
dtype: bool

No missing data!

## Processing Data

Since price comparison will be an important part of my research question it will be useful to add an *Average Cost for two* column in CAD to allow for easier comparison. The countries in our list and their currencies are as follows:

In [119]:
#Currencies
print(cleaned_df['Currency'].unique())
#Countries
print(cleaned_df.index.unique(0))

['Botswana Pula(P)' 'Brazilian Real(R$)' 'Dollar($)' 'Emirati Diram(AED)'
 'Indian Rupees(Rs.)' 'Indonesian Rupiah(IDR)' 'NewZealand($)'
 'Pounds(Ć£)' 'Qatari Rial(QR)' 'Rand(R)' 'Sri Lankan Rupee(LKR)'
 'Turkish Lira(TL)']
Index(['Phillipines', 'Brazil', 'United States', 'Australia', 'Canada',
       'Singapore', 'UAE', 'India', 'Indonesia', 'New Zealand',
       'United Kingdom', 'Qatar', 'South Africa', 'Sri Lanka', 'Turkey'],
      dtype='object', name='Country')


Interestingly, all currencies match their respective country except for the Botswana Pula which is being used for the Phillipines instead of the Philippine Peso. However, this shouldn't matter much since we are converting it to CAD anyways. Since conversion rates are always fluctating we will use the average conversion rate from 2017 - 2020 years provided by the [Bank of Canada](https://www.bankofcanada.ca/rates/exchange/annual-average-exchange-rates/)


In [202]:
#Reading in the exchange rate data
import json
#data = json.load(open('..\\data\\raw\\FX_RATES_ANNUAL-sd-2017-01-01.json'))
#df_1 = pd.DataFrame(data)
#df_2 = pd.DataFrame(data["observations"])

with open('..\\data\\raw\\FX_RATES_ANNUAL-sd-2017-01-01.json', "r") as read_file:
    dict = json.load(read_file)

df_1 = pd.DataFrame.from_dict(dict['seriesDetail']).T
df_1

,label,description,dimension
FXAAUDCAD,AUD/CAD,Australian dollar to Canadian dollar annual ex...,"{'key': 'd', 'name': 'date'}"
FXABRLCAD,BRL/CAD,Brazilian real to Canadian dollar annual excha...,"{'key': 'd', 'name': 'date'}"
FXACNYCAD,CNY/CAD,Chinese renminbi to Canadian dollar annual exc...,"{'key': 'd', 'name': 'date'}"
FXAEURCAD,EUR/CAD,European euro to Canadian dollar annual exchan...,"{'key': 'd', 'name': 'date'}"
FXAHKDCAD,HKD/CAD,Hong Kong dollar to Canadian dollar annual exc...,"{'key': 'd', 'name': 'date'}"
FXAINRCAD,INR/CAD,Indian rupee to Canadian dollar annual exchang...,"{'key': 'd', 'name': 'date'}"
FXAIDRCAD,IDR/CAD,Indonesian rupiah to Canadian dollar annual ex...,"{'key': 'd', 'name': 'date'}"
FXAJPYCAD,JPY/CAD,Japanese yen to Canadian dollar annual exchang...,"{'key': 'd', 'name': 'date'}"
FXAMYRCAD,MYR/CAD,Malaysian ringgit to Canadian dollar annual ex...,"{'key': 'd', 'name': 'date'}"
FXAMXNCAD,MXN/CAD,Mexican peso to Canadian dollar annual exchang...,"{'key': 'd', 'name': 'date'}"


In [204]:
df_2 = pd.DataFrame.from_dict(dict["observations"]).T
df_2

,0,1,2,3
d,2017-01-01,2018-01-01,2019-01-01,2020-01-01
FXAAUDCAD,{'v': '0.9951'},{'v': '0.9687'},{'v': '0.9228'},{'v': '0.9247'}
FXABRLCAD,{'v': '0.4071'},{'v': '0.3566'},{'v': '0.3371'},{'v': '0.2625'}
FXACNYCAD,{'v': '0.1921'},{'v': '0.1961'},{'v': '0.1922'},{'v': '0.1944'}
FXAEURCAD,{'v': '1.4650'},{'v': '1.5302'},{'v': '1.4856'},{'v': '1.5298'}
FXAHKDCAD,{'v': '0.1667'},{'v': '0.1653'},{'v': '0.1693'},{'v': '0.1730'}
FXAINRCAD,{'v': '0.01995'},{'v': '0.01897'},{'v': '0.01885'},{'v': '0.01810'}
FXAIDRCAD,{'v': '0.000097'},{'v': '0.000091'},{'v': '0.000094'},{'v': '0.000092'}
FXAJPYCAD,{'v': '0.01158'},{'v': '0.01174'},{'v': '0.01217'},{'v': '0.01257'}
FXAMYRCAD,{'v': '0.3020'},{'v': '0.3213'},{'v': '0.3203'},NaN
